# Your First (But Admittedly Boring) Neural Network

Shamelessly copied from the PyTorch example.

# First, let's import some stuff we'll need

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

# Set up some functions we need (but you don't need to understand them now)

In [2]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.fc1(x.view((-1, 28 * 28)))
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [5]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [6]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Get the data and do some preprocessing

In [7]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])


train_ds = datasets.MNIST('../data', train=True, download=True,
                   transform=transform)
valid_ds = datasets.MNIST('../data', train=False,
                   transform=transform)

train_dl = torch.utils.data.DataLoader(train_ds, batch_size = 32)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size = 32)

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# Making sure everything has the right shape

In [13]:
train_ds[0][0].shape

train_ds[0][0].flatten(start_dim=1, end_dim=2).shape
train_ds[0][0][0].numpy()

array([[-0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296],
       [-0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296],
       [-0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.42421296, -0.42421296, -0.42421296, -0.42421296,
        -0.42421296, -0.4242

# Initializing the model and optimizer

In [14]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

epochs = 10

device = torch.device('cpu')

# Training time!

In [10]:
for epoch in range(1, epochs):
    train(model, device, train_dl, optimizer, epoch)
    test(model, device, valid_dl)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.335682
Train Epoch: 1 [3200/60000 (5%)]	Loss: 0.544793
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.406264
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.261447
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.312997
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.396153
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.255245
Train Epoch: 1 [22400/60000 (37%)]	Loss: 0.232460
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.301933
Train Epoch: 1 [28800/60000 (48%)]	Loss: 0.320268
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.644341
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.208651
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.317742
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.448457
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.294242
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.264078
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.586832
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.326815
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.350068

Test set: Average loss: 0.4017, Accuracy: 8836/10000 (88%)

#Predictions

In [15]:
model(train_ds[0][0])

tensor([[-2.0384, -2.7691, -2.6889, -2.3626, -2.1093, -2.6008, -3.2610, -1.8211,
         -2.2925, -1.9094]], grad_fn=<LogSoftmaxBackward>)